In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

In [3]:
ames_df = pd.read_csv('train.csv')

Let's divide the dataframe into continuous and categorical dataframes.

In [28]:
int_df = ames_df.select_dtypes('number')

def uniq_col(df):
    cols = int_df.columns
    keep = []
    for col in cols:
        leng = len(list(df[col].unique()))
        if(leng > 100):
            keep.append(col)
    return keep

cont_df = uniq_col(int_df)

cont_df = int_df[[df for df in cont_df]]

cont_df.drop(['Id'], axis =1 ,inplace = True)


C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categorical = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames_df[continuous]

log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

def normalize(feature):
    return (feature - feature.mean()/feature.std())

ames_log_norm = ames_log.apply(normalize)

#one hot encoded categoricals
ames_ohe = pd.get_dummies(ames_df[categorical], prefix = 'categoricals', drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis = 1)

In [36]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [40]:
X = preprocessed.drop(['SalePrice_log'], axis = 1)
Y = preprocessed['SalePrice_log']

result = stepwise_selection(X, Y, verbose = True)

C:\Users\Public\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Public\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Public\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Public\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Public\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp i

Add  GrLivArea_log                  with p-value 1.59847e-243
Add  categoricals_TA                with p-value 1.56401e-67
Add  1stFlrSF_log                   with p-value 7.00069e-48
Add  categoricals_Fa                with p-value 1.70471e-37
Add  categoricals_OldTown           with p-value 3.20105e-23
Add  categoricals_Gd                with p-value 4.12635e-21
Add  categoricals_Edwards           with p-value 9.05184e-17
Add  categoricals_IDOTRR            with p-value 1.10068e-18
Add  LotArea_log                    with p-value 1.71728e-13
Add  categoricals_NridgHt           with p-value 7.05633e-12
Add  categoricals_Duplex            with p-value 4.30647e-11
Add  categoricals_NAmes             with p-value 2.25803e-09
Add  categoricals_SWISU             with p-value 5.40743e-09
Add  categoricals_BrkSide           with p-value 8.79638e-10
Add  categoricals_Sawyer            with p-value 6.92011e-09
Add  categoricals_NoRidge           with p-value 5.87105e-08
Add  categoricals_Somer

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Add  categoricals_Mitchel           with p-value 0.00994666
Drop categoricals_Somerst           with p-value 0.0500753
Add  categoricals_SawyerW           with p-value 0.00427685


In [41]:
result

['GrLivArea_log',
 'categoricals_TA',
 '1stFlrSF_log',
 'categoricals_Fa',
 'categoricals_OldTown',
 'categoricals_Gd',
 'categoricals_Edwards',
 'categoricals_IDOTRR',
 'LotArea_log',
 'categoricals_NridgHt',
 'categoricals_Duplex',
 'categoricals_NAmes',
 'categoricals_SWISU',
 'categoricals_BrkSide',
 'categoricals_Sawyer',
 'categoricals_NoRidge',
 'categoricals_StoneBr',
 'categoricals_MeadowV',
 'categoricals_New',
 'categoricals_BrDale',
 'categoricals_RM',
 'categoricals_RL',
 'categoricals_FV',
 'categoricals_RH',
 'categoricals_NWAmes',
 'categoricals_Mitchel',
 'categoricals_SawyerW']

In [43]:
import statsmodels.api as sm

x_fin = X[result]
x_with_intercept = sm.add_constant(x_fin)
model = sm.OLS(Y,x_with_intercept).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     269.0
Date:                Fri, 24 Apr 2020   Prob (F-statistic):               0.00
Time:                        18:41:56   Log-Likelihood:                 585.39
No. Observations:                1460   AIC:                            -1115.
Df Residuals:                    1432   BIC:                            -966.8
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -8.2889      0.286    -29.031      0.000      -8.849      -7.729
GrLivArea_log            0.4423      0.018     24.477      0.000       0.407       0.478
categoricals_TA         -0.2804      0.022    -12.859      0.000      -0.323      -0.238
1stFlrSF_log             0.1816      0.019      9.645      0.000       0.145       0.219
categoricals_Fa         -0.4143      0.035    -11.864      0.000      -0.483      -0.346
categoricals_OldTown    -0.3445      0.025    -13.615      0.000      -0.394      -0.295
categoricals_Gd         -0.1606      0.020     -8.046      0.000      -0.200      -0.121
categoricals_Edwards    -0.2804      0.019    -14.530      0.000      -0.318      -0.243
categoricals_IDOTRR     -0.3429      0.039     -8.855      0.000      -0.419      -0.267
LotArea_log              0.0846      0.011      7.387      0.000       0.062       0.107
categoricals_NridgHt     0.1539      0.023      6.809      0.000       0.110       0.198
categoricals_Duplex     -0.1627      0.024     -6.678      0.000      -0.210      -0.115
categoricals_NAmes      -0.1503      0.015     -9.981      0.000      -0.180      -0.121
categoricals_SWISU      -0.2502      0.036     -7.020      0.000      -0.320      -0.180
categoricals_BrkSide    -0.2253      0.027     -8.493      0.000      -0.277      -0.173
categoricals_Sawyer     -0.1608      0.022     -7.342      0.000      -0.204      -0.118
categoricals_NoRidge     0.1736      0.028      6.221      0.000       0.119       0.228
categoricals_StoneBr     0.1813      0.035      5.226      0.000       0.113       0.249
categoricals_MeadowV    -0.2645      0.047     -5.592      0.000      -0.357      -0.172
categoricals_New         0.0592      0.017      3.388      0.001       0.025       0.094
categoricals_BrDale     -0.1891      0.049     -3.839      0.000      -0.286      -0.092
categoricals_RM          0.4322      0.059      7.363      0.000       0.317       0.547
categoricals_RL          0.3961      0.062      6.356      0.000       0.274       0.518
categoricals_FV          0.4814      0.066      7.284      0.000       0.352       0.611
categoricals_RH          0.3397      0.076      4.490      0.000       0.191       0.488
categoricals_NWAmes     -0.0821      0.021     -3.837      0.000      -0.124      -0.040
categoricals_Mitchel    -0.0776      0.026     -3.004      0.003      -0.128      -0.027
categoricals_SawyerW    -0.0665      0.023     -2.862      0.004      -0.112      -0.021
==============================================================================
Omnibus:                      295.535   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1270.571
Skew:                          -0.903   Prob(JB):                    1.26e-276
Kurtosis

In [45]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(X,Y.values.ravel())
selector.support_

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [46]:
selected_columns = X.columns[selector.support_]
linreg.fit(X[selected_columns], Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [55]:
yhat = linreg.predict((X[selected_columns]))

SS_residual = np.sum((Y-yhat)**2)
SS_total = np.sum((Y-np.mean(Y))**2)
r_squared = 1 - (float(SS_residual))/SS_total
adjusted_r_squared = 1 - (1-r_squared)*(len(Y)-1)/(len(Y)-X[selected_columns].shape[1]-1)

In [56]:
r_squared

0.6149723925230821

In [57]:
adjusted_r_squared

0.6136483636115384

In [59]:
X.columns[selector.support_]

Index(['GrLivArea_log', 'categoricals_FV', 'categoricals_RH',
       'categoricals_RL', 'categoricals_RM'],
      dtype='object')